# Imports

In [1]:
from OPqkdsim.channel import *
from OPqkdsim.measure import *
from OPqkdsim.polarization import *
from OPqkdsim.source import *
from OPqkdsim.timeline import *
from OPqkdsim.random import *

In [2]:
# Initialize Timeline

timeline = Timeline()
shots=10
# Define bit lists and modulation lists
rng = CRNG(method="crypto")
alice_bit_list = rng.generate_bits(shots)
alice_modulation_list = rng.generate_basis(shots)
bob_modulation_list = rng.generate_basis(shots)
bob_bit_list = [] 

In [3]:
# Initialize Alice Components

laser = DFBLaser(timeline, shots=shots)
pc1 = PolarizationController(timeline)  # Alice's polarization control
pm1 = AlicePolarizationModulator(timeline, modulation_list=alice_modulation_list, bit_list=alice_bit_list)
attenuator = VariableOpticalAttenuator(timeline, attenuation_dB=60)  # Attenuates to single-photon level # why 60db ?

# Quantum channel

channel = QuantumChannel(timeline)  


In [4]:
# Initialize Bob Components

pc2 = PolarizationController(timeline, compensate=True)  # Bob's first polarization control
pm2 = BobPolarizationModulator(timeline, modulation_list=bob_modulation_list)
pc3 = PolarizationController(timeline)  # Final polarization correction

# Create Detectors

detector_H = SinglePhotonDetector(timeline, name="Detector_H", bit_list=bob_bit_list)
detector_V = SinglePhotonDetector(timeline, name="Detector_V", bit_list=bob_bit_list)
detector_D_plus = SinglePhotonDetector(timeline, name="Detector_D+", bit_list=bob_bit_list)
detector_D_minus = SinglePhotonDetector(timeline, name="Detector_D-", bit_list=bob_bit_list)

# Create PBS

pbs = PolarizationBeamSplitter(timeline, detector_H, detector_V, detector_D_plus, detector_D_minus, bob_modulation_list)


In [5]:
# Connect Components Using Timeline’s Publish-Subscribe Model

timeline.subscribe(laser, pc1.process_signal)
timeline.subscribe(pc1, pm1.process_signal)
timeline.subscribe(pm1, attenuator.process_signal)
timeline.subscribe(attenuator, channel.propagate_signal)
timeline.subscribe(channel, pc2.process_signal)
timeline.subscribe(pc2, pm2.process_signal)
timeline.subscribe(pm2, pc3.process_signal)
timeline.subscribe(pc3, pbs.process_signal)
timeline.subscribe(pbs, detector_H.detect_photon)
timeline.subscribe(pbs, detector_V.detect_photon)
timeline.subscribe(pbs, detector_D_plus.detect_photon)
timeline.subscribe(pbs, detector_D_minus.detect_photon)

In [6]:
# Start Simulation by Emitting Light from the Laser
laser.initialize_laser()
timeline.execute_events()

[1.74033e+09 s] Pulse emitted at 1.55e-06 m, Power: 1.20000e-07 W
[1.74033e+09 s] Polarization Adjusted: Ex=1.09631e-05, Ey=4.44409e-08
[1.74033e+09 s] Phase Modulated: Basis=1, Bit=1, φ=-1.57080 rad
[1.74033e+09 s] VOA applied 60.00000 dB attenuation. New Power: 1.20000e-13 W
[1.740e+09 s] Signal propagated: P_out=4.50e-51 W
[1.74033e+09 s] Polarization Adjusted: Ex=2.12310e-27, Ey=-2.11525e-30
[1.74033e+09 s] Basis: 1, Applied Phase Shift: 0.78540 rad
[1.74033e+09 s] Polarization Adjusted: Ex=2.12195e-27, Ey=-2.11410e-30
[1.740e+09 s] PBS Output - P_D+: 2.25e-51 W, P_D-: 2.25e-51 W


IndexError: invalid index to scalar variable.

In [ ]:
matching_bits = []
for alice_basis, bob_basis, alice_bit, bob_bit in zip(alice_modulation_list, bob_modulation_list, alice_bit_list, bob_bit_list):
    if alice_basis == bob_basis:  # Matching bases
        matching_bits.append(alice_bit)

# Display Results
print("\n--- BB84 Protocol Results ---")
print(f"Alice's Bits: {alice_bit_list}")
print(f"Alice's Bases: {alice_modulation_list}")
print(f"Bob's Bases: {bob_modulation_list}")
print(f"Bob's Received Bits: {bob_bit_list}")
print(f"Final Key (Matching Basis): {matching_bits}")


--- BB84 Protocol Results ---
Alice's Bits: []
Alice's Bases: []
Bob's Bases: []
Bob's Received Bits: []
Final Key (Matching Basis): []
